# Install packages

In [ ]:
# # 한번씩 실행해서 neo4j 패키지 설치해주세요
!python -m pip install neo4j
!pip install --upgrade google-cloud-BigQuery
!pip install gspread
!pip install --upgrade oauth2client
!pip install db-dtypes
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 4.3 MB/s 
ERROR: Operation cancelled by user
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/cmdoptions.py", line 23, in <module>
    from pip._internal.cli.parser import ConfigOptionParser
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/parser.py", li

# Modules

In [ ]:
# Basics
import pandas as pd
import numpy as np
import json
import logging
import os
import pickle
from IPython.display import display
import time

## Neo4j
from flask import (Flask, g, request, Response)
from neo4j import (GraphDatabase, basic_auth)

## Spreadsheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials

## BigQuery
from google.cloud import bigquery

## recommendation
import surprise
from sklearn.model_selection import train_test_split
from surprise import Dataset
from surprise import Reader
from surprise import SVD

pd.set_option("display.max_columns", None)

In [ ]:
## prepare the local DBMS
# 아래 코드는 비밀번호말고 전혀 수정할 내용 없음
scheme = "bolt" 
host_name = "localhost"
port = 7687
url = f"{scheme}://{host_name}:{port}"
user = "neo4j"

# 지금 내가 실행한 DB의 비밀번호를 입력해주자.
password = "0000"

driver = GraphDatabase.driver(url, auth=(user, password))
# driver.verify_connectivity()

driver.session(database="neo4j") # 지금 start시킨 database 이름

In [ ]:
!export GOOGLE_APPLICATION_CREDENTIALS = '/Users/sungwoo/Desktop/Computer/bkms/bkms-bq-956d2fdf3af7.json'

## 여기에도 json 파일의 주소를 붙여넣습니다.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/sungwoo/Desktop/Computer/bkms/bkms-bq-956d2fdf3af7.json"
client = bigquery.Client()
project = client.project

zsh:1: bad assignment


In [ ]:
## Import crawling data
df = pd.read_csv("./ratings.csv", engine="python")
id_info = pd.read_csv("./id_info.csv", engine="python")

# Upload existing data

## Neo4j

In [ ]:
# Rating column 전처리 & nan 결측값 제거
# 여기서 nan 값 제거 안하면, 추후 삽입 과정에서 오류 발생
weird_rows = []
for i in range(df.shape[0]):
    try:
        df['rating'][i] = float(df['rating'][i])
    except:
#         print("As there's a weird value in 'rating' column, the corresponding row data should be deleted.")
#         print(f"Row index : {i}")
        weird_rows.append(i)
#         print("=====================================================================================")

df = df.iloc[list(set(range(df.shape[0]))-set(weird_rows)), :]
df['rating'] = pd.to_numeric(df['rating'])
df = df.dropna()
df = df.reset_index(drop=True)
# df = df.iloc[:,2:]

/var/folders/h_/ptp7_f_n1qd09d3cd2gz5dbw0000gn/T/ipykernel_11233/2836533816.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'][i] = float(df['rating'][i])


In [ ]:
# for API
genre_dict_API = {1:"Biography", 2:"Film Noir", 3:"Game Show", 4:"Musical", 5:"Sport", 6:"Short", 7:"Adult", 
                  12:"Adventure", 14:"Fantasy", 16:"Animation", 18:"Drama", 27:"Horror", 28:"Action", 35:"Comedy",
                  36:"History", 37:"Western", 53:"Thriller", 80:"Crime", 99:"Documentary", 878:"Science Fiction",
                  9648:"Mystery", 10402:"Music", 10749:"Romance", 10751:"Family", 10752:"War", 10763:"News",
                  10764:"Reality", 10767:"Talk Show"}

In [ ]:
## create movie

with driver.session(database="neo4j") as session:
    def create_movie(tx, movie_watchaID, movie_year, movie_runtime, movie_title, movie_genre, movie_countries, movie_age, movie_lan):
        query = (
            "CREATE (m1:Movie { id: $movie_watchaID, year: $movie_year, runtime: $movie_runtime, title: $movie_title, genre : $movie_genre, countries : $movie_countries ,age : $movie_age, original_lan : $movie_lan}) "
            "RETURN m1"
        )
        result = tx.run(query, movie_watchaID=movie_watchaID, movie_year=movie_year, movie_runtime=movie_runtime, movie_title=movie_title, movie_genre=movie_genre, movie_countries=movie_countries, movie_age=movie_age, movie_lan=movie_lan)
        return [{"m1": record["m1"]["movie_title"]} for record in result]
    
    visited = {}
    for i in range(id_info.shape[0]):
        movie_watchaID = str(id_info.loc[i, "watcha_movie_id"])
        visited[movie_watchaID] = 0
        
    for i in range(id_info.shape[0]):
        movie_watchaID = str(id_info.loc[i, "watcha_movie_id"])
        visited[movie_watchaID] += 1
        if visited[movie_watchaID] == 1:
            movie_year = str(id_info.loc[i, "year"])
            movie_runtime = str(id_info.loc[i, "runtime"])
            movie_title = id_info.loc[i, "title"]
            movie_genre = genre_dict_API[eval(id_info.loc[i, "genres"])[0]]
            try:
                movie_countries = eval(id_info.loc[i, "countries"])[0]
            except:
                movie_countries = 'None'
            movie_age = str(id_info.loc[i, "age"])
            movie_lan = id_info.loc[i, "originalLanguage"]
            result = session.write_transaction(create_movie, movie_watchaID=movie_watchaID, movie_year=movie_year, movie_runtime=movie_runtime, movie_title=movie_title, movie_genre=movie_genre, movie_countries=movie_countries, movie_age=movie_age, movie_lan=movie_lan)
#             print(f"{i}th iteration")
#     print("Finished")

In [ ]:
## create user
with driver.session(database="neo4j") as session:
    def create_user(tx, movie_watchaID, user_watchaID, like_movie):
        query = (
            "MATCH (m1:Movie { id: $movie_watchaID }) "
            "CREATE (u1:User { id: $user_watchaID }) "
            "CREATE (u1)-[:LIKE {Likeness: $like_movie}]->(m1) "
            "RETURN m1, u1"
        )
        result = tx.run(query, movie_watchaID=movie_watchaID, user_watchaID=user_watchaID, like_movie=like_movie)
        return [{"m1": record["m1"]["movie_title"]} for record in result]
    
    visited = {}
    for i in range(df.shape[0]):
        user_watchaID = str(df.loc[i, "user_id"])
        visited[user_watchaID] = 0
    for i in range(df.shape[0]):
        movie_watchaID = str(df.loc[i, "watcha_movie_id"])
        user_watchaID = str(df.loc[i, "user_id"])
        like_movie = str(df.loc[i, "rating"])
        visited[user_watchaID] += 1
        if visited[user_watchaID] == 1:
            result = session.write_transaction(create_user, movie_watchaID=movie_watchaID, user_watchaID=user_watchaID, like_movie=like_movie)
#             print(f"=========== {i}th iteration ===========")
#     print('FINISHED')

In [ ]:
## create actor
with driver.session(database="neo4j") as session:
    def create_actor(tx, actor_name):
        query = (
            "CREATE (a1:Actor { name: $actor_name }) "
            "RETURN a1"
        )
        result = tx.run(query, actor_name=actor_name)
        return [{"a1": record["a1"]["actor_name"]} for record in result]
    
    user_actors = []
    for i in range(df.shape[0]):
        objective = eval(df['Favorite_Actors'][i])
        for j in range(len(objective)):
            if j%3 == 0:
                user_actors.append(objective[j])
    user_actors = list(set(user_actors))
    for t in range(len(user_actors)):
#         print(f"=========== {t}th iteration ===========")
        last = t
        result = session.write_transaction(create_actor, actor_name=user_actors[t])
    
    actors = []
    for i in range(id_info.shape[0]):
        movie_watchaID = str(id_info.loc[i, "watcha_movie_id"])
        movie_actors = eval(id_info.loc[i, "cast"])
        for actor in movie_actors:
            actors.append(actor)
    actors = list(set(actors)-set(user_actors))
    for t in range(len(actors)):
        result = session.write_transaction(create_actor, actor_name=actors[t])
#         print(f"=========== {last+t}th iteration ===========")
    
#     print('FINISHED')

In [ ]:
## create preferred actors
with driver.session(database="neo4j") as session:
    def create_likeactor(tx, user_watchaID, actor_name, like_actor):
        query = (
            "MATCH (u1:User { id: $user_watchaID }) "
            "MATCH (a1:Actor { name: $actor_name }) "
            "CREATE (u1)-[:LIKE {Likeness: $like_actor}]->(a1) "
            "RETURN a1, u1"
        )
        result = tx.run(query, user_watchaID=user_watchaID, actor_name=actor_name, like_actor=like_actor)
        return [{"a1": record["a1"]["actor_name"]} for record in result]
    
    visited = {}
    for i in range(df.shape[0]):
        user_watchaID = str(df.loc[i, "user_id"])
        visited[user_watchaID] = 0
    
    for i in range(df.shape[0]):
        user_watchaID = df['user_id'][i]
        user_actors = []
        user_scores = []
        like = {}
        visited[user_watchaID] += 1
        if visited[user_watchaID] == 1:
#             print(f"=========== {i}th iteration ===========")
            objective = eval(df['Favorite_Actors'][i])
            for j in range(len(objective)):
                if j%3 == 0:
                    user_actors.append(objective[j])
                elif j%3 == 1:
                    user_scores.append(objective[j][7:9])
            for j in range(len(user_scores)):
                like[user_actors[j]] = user_scores[j]
                result = session.write_transaction(create_likeactor, user_watchaID=user_watchaID, actor_name=user_actors[j], like_actor=like[user_actors[j]])
#     print('FINISHED')

In [ ]:
## create directors
with driver.session(database="neo4j") as session:
    def create_director(tx, director_name):
        query = (
            "CREATE (d1:Director { name: $director_name }) "
            "RETURN d1"
        )
        result = tx.run(query, director_name=director_name)
        return [{"d1": record["d1"]["name"]} for record in result]
    
    user_directors = []
    for i in range(df.shape[0]):
        objective = eval(df['Favorite_Directors'][i])
        for j in range(len(objective)):
            if j%3 == 0:
                user_directors.append(objective[j])
    user_directors = list(set(user_directors))
    for t in range(len(user_directors)):
#         print(f"=========== {t}th iteration ===========")
        last = t
        result = session.write_transaction(create_director, director_name=user_directors[t])
    
    directors = []
    for i in range(id_info.shape[0]):
        movie_watchaID = str(id_info.loc[i, "watcha_movie_id"])
        movie_directors = eval(id_info.loc[i, "significants"])
        for director in movie_directors:
            directors.append(director)
    directors = list(set(directors)-set(user_directors))
    for t in range(len(directors)):
        result = session.write_transaction(create_director, director_name=directors[t])
#         print(f"=========== {last+t}th iteration ===========")
    
#     print('FINISHED')

In [ ]:
## create preferred directors
with driver.session(database="neo4j") as session:
    def create_likedirector(tx, user_watchaID, director_name, like_director):
        query = (
            "MATCH (u1:User { id: $user_watchaID }) "
            "MATCH (d1:Director { name: $director_name }) "
            "CREATE (u1)-[:LIKE {Likeness: $like_director}]->(d1) "
            "RETURN d1, u1"
        )
        result = tx.run(query, user_watchaID=user_watchaID, director_name=director_name, like_director=like_director)
        return [{"d1": record["d1"]["director_name"]} for record in result]
    
    
    visited = {}
    for i in range(df.shape[0]):
        user_watchaID = str(df.loc[i, "user_id"])
        visited[user_watchaID] = 0
    
    for i in range(df.shape[0]):
        user_watchaID = df['user_id'][i]
        user_directors = []
        user_scores = []
        like = {}
        visited[user_watchaID] += 1
        if visited[user_watchaID] == 1:
#             print(f"=========== {i}th iteration ===========")
            objective = eval(df['Favorite_Directors'][i])
            for j in range(len(objective)):
                if j%3 == 0:
                    user_directors.append(objective[j])
                elif j%3 == 1:
                    user_scores.append(objective[j][7:9])
            for j in range(len(user_scores)):
                like[user_directors[j]] = user_scores[j]
                result = session.write_transaction(create_likedirector, user_watchaID=user_watchaID, director_name=user_directors[j], like_director=user_scores[j])
#     print('FINISHED')

In [ ]:
## create play relationship
with driver.session(database="neo4j") as session:
    def create_play(tx, movie_watchaID, actor_name):
        query = (
            "MATCH (a1:Actor { name: $actor_name }) "
            "MATCH (m1:Movie { id: $movie_watchaID }) "
            "MERGE (a1)-[:PLAY]->(m1) "
            "RETURN a1, m1"
        )
        result = tx.run(query, movie_watchaID=movie_watchaID, actor_name=actor_name)
        return [{"a1": record["a1"]["actor_name"]} for record in result]
    
    for i in range(id_info.shape[0]):
#         print(f"=========== {i}th iteration ===========")
        movie_watchaID = id_info['watcha_movie_id'][i]
        movie_cast = eval(id_info['cast'][i])
        for j in range(len(movie_cast)):
            result = session.write_transaction(create_play, movie_watchaID=movie_watchaID, actor_name=movie_cast[j])
#     print('FINISHED')

In [ ]:
## create direct relationship
with driver.session(database="neo4j") as session:
    def create_direct(tx, movie_watchaID, director_name):
        query = (
            "MATCH (d1:Director { name: $director_name }) "
            "MATCH (m1:Movie { id: $movie_watchaID }) "
            "MERGE (d1)-[:DIRECT]->(m1) "
            "RETURN d1, m1"
        )
        result = tx.run(query, movie_watchaID=movie_watchaID, director_name=director_name)
        return [{"d1": record["d1"]["name"]} for record in result]
    
    for i in range(id_info.shape[0]):
#         print(f"=========== {i}th iteration ===========")
        movie_watchaID = id_info['watcha_movie_id'][i]
        movie_director = eval(id_info['significants'][i])
        for j in range(len(movie_director)):
            result = session.write_transaction(create_direct, movie_watchaID=movie_watchaID, director_name=movie_director[j])
#     print('FINISHED')

## BigQuery

* reference: [링크](https://blog.coupler.io/how-to-crud-bigquery-with-python/#Connecting_Python_to_BigQuery)
* BigQuery 서비스 계정 만들기: [링크](https://cloud.google.com/docs/authentication/getting-started)

### CREATE Dataset and Table

In [ ]:
## dataset이 있으면 불러오고, 없으면 같은 이름의 dataset을 생성
## 아래 dataset_id에 사용할 dataset name을 입력해주면 됩니다.

dataset_id = "bkms_final"
datasets = [item.dataset_id for item in list(client.list_datasets())]
if dataset_id in datasets:
    print(f"{dataset_id} already exists.")
    dataset = client.get_dataset(dataset_id)  # Make an API request.
else:
    # create dataset
    print(f"{dataset_id} does not exist. Creating {dataset_id}...")
    dataset_ref = bigquery.DatasetReference.from_string(f"{project}.{dataset_id}", default_project=client.project)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    dataset = client.create_dataset(dataset)
    print(f"Successfully created {dataset_id}!")

bkms_final does not exist. Creating bkms_final...
Successfully created bkms_final


In [ ]:
## table이 있으면 불러오고, 없으면 같은 이름의 table을 생성
## 아래 table_name에 사용할 table name을 입력해주면 됩니다.

table_name = "final_movie"
table_id = f"{project}.{dataset_id}.{table_name}"
tables = [item.table_id for item in client.list_tables(dataset=f"{project}.{dataset_id}")]
if table_name in tables:
    print(f"{table_name} already exists.")
    table = client.get_table(table_id)
else:
    ## create table
    print(f"{table_name} does not exist. Creating {table_name}...")
    to_create = bigquery.Table.from_string(table_id)
    schema = [
        bigquery.SchemaField("watcha_movie_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("title", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("user_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("rating", "FLOAT"),
    ]
    table = bigquery.Table(to_create, schema=schema)
    table = client.create_table(table)
    print(f"Successfully created {table_name}!")

final_movie does not exist. Creating final_movie...
Successfully created final_movie!


### Data Import and Load

In [ ]:
toUse = df[["watcha_movie_id", "title", "user_id", "rating"]].copy()
toUse["rating"] = toUse["rating"].astype(np.float16)

rows_to_insert = []
for i in range(toUse.shape[0]):
    row = {col:str(toUse[col].iloc[i]) for col in toUse.columns.values}
    rows_to_insert.append(row)
    
json_obj = json.dumps(rows_to_insert)

## insert rows
errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
if errors == []:
    print("New rows have been added.")
else:
    print("Encountered errors while inserting rows: {}".format(errors))

New rows have been added.
